# Donkey Colab 3.0.2 with GPU(TensorFlow 2.0.0 Beta1)

Donkey Colab with GPU.
- Version 0.1(2019年7月16日)
- Version 0.1(2019年7月12日)

# TensorFlowを2.0.0 Beta1にUpdate

TensorFlowのVersionは、DonkeyCarのRaspberryPi3にはいっているバージョンと同じバージョンを構築します。

In [0]:
!pip install tensorflow-gpu==2.0.0-Beta1

# GPUの割り振りを確認

"Found GPU at: /device:GPU:0"と表示されれば、GPUが使用できる状態です。

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# DonkeyレポジトリをGit clone

In [0]:
!git clone https://github.com/FaBoPlatform/donkeycar donkey

# インストール

In [0]:
%cd /content/
!pip3 install -e donkey

# Projectの作成

In [0]:
!donkey createcar --path /content/mycar

# datas.zipのアップロードとデータの解凍
下記コードを実行すると、ファイルのアップロード フォームが出現します。datas.zipをアップリードします。

datas.zipは、/content以下にアップされます。mv コマンドで、/content/mycar/dataに移動し、解凍します。

In [0]:
import os
from google.colab import files

if(os.path.exists("/content/datas.zip")):
   os.remove("/content/datas.zip")
if(os.path.exists("/content/mycar/data/datas.zip")):
   os.remove("/content/mycar/data/datas.zip")
   
uploaded = files.upload()

WORK_FOLDER = "/content/mycar/data/"
if(os.path.exists(WORK_FOLDER) == False):
  os.makedirs(WORK_FOLDER)

!mv /content/datas.zip /content/mycar/data/
%cd /content/mycar/data/
!unzip -o datas.zip

# ランタイムの再起動

Colabメニューの**[ランタイム]-[ランタイムの再起動]**して、ライブラリを取り込みます。

![代替テキスト](https://faboplatform.github.io/DonkeyDocs/0.DonkeyCar3%E3%81%AE%E6%A7%8B%E7%AF%89/img/co001.png)

![代替テキスト](https://faboplatform.github.io/DonkeyDocs/0.DonkeyCar3%E3%81%AE%E6%A7%8B%E7%AF%89/img/co002.png)

# 学習用のScriptをImport

DonkeyCarの学習用のScriptを読み込みます。


In [0]:
#!/usr/bin/env python3

import os

from docopt import docopt
import donkeycar as dk
import tensorflow as tf
from donkeycar.templates.train import multi_train
from donkeycar.parts.transform import Lambda
from donkeycar.parts.keras import KerasLinear
from donkeycar.parts.actuator import PCA9685, PWMSteering, PWMThrottle
from donkeycar.parts.datastore import TubGroup, TubWriter

from donkeycar.parts.datastore import TubGroup, TubWriter
from donkeycar.parts.keras import KerasLinear
from donkeycar.parts.transform import Lambda



#学習の開始

ブラウザの右側のWindowの[目次]-[コードスニペット]-[ファイル]のタブから[ファイル]タブを選択し、mycar/data以下にアップロードされたフォルダ名を確認します。

![代替テキスト](https://faboplatform.github.io/DonkeyDocs/0.DonkeyCar3%E3%81%AE%E6%A7%8B%E7%AF%89/img/co003.png)

下記学習例では、model_typeをcategoricalに指定しているので、DonkeyCar側 RaspberryPi3の~/mycar/myconfig.py のDEFAULT_MODEL_TYPEもコメントアウトを外し、categoricalに指定します。


~/mycar/myconfig.py(DonkeyCar)


---

```
DEFAULT_MODEL_TYPE="categotical"
```

---




In [0]:
import tensorflow as tf
with tf.device('/gpu:0'):
  cfg = dk.load_config()
  # tubはアップロードしたフォルダ名を配列形式で指定(書き換え必須)
  # 下記では、/content/mycar/data/tub_4_19-07-12と/content/mycar/data/tub_6_19-07-12の2つを学習対象にしている
  tub = ["/content/mycar/data/tub_4_19-07-12","/content/mycar/data/tub_6_19-07-12"] 
  model =  "/content/mycar/models/mypilot.h5"
  transfer = None
  # model typeもRaspberryPi3側と合わせる必要がある
  model_type = "categorical"  # linear, categorical, rnn, imu, behavior, 3dを指定できます
  continuous = None
  aug = None
    
  multi_train(cfg, tub, model, transfer, model_type, continuous, aug)


# 学習結果の転送
学習結果は、/content/mycar/models/mypilot.h5に保存されます。
mypilotをダブルクリックしローカルにダウンロードし、SSHでRaspberryPiの~/mycar/models/mypilotにコピーします。

![](https://faboplatform.github.io/DonkeyDocs/0.DonkeyCar3%E3%81%AE%E6%A7%8B%E7%AF%89/img/upload.png)